# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'09-02-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'09-02-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-09-03 04:28:22,33.93911,67.709953,38243,1409,29315,7519.0,Afghanistan,98.239481,3.684334
1,NaN,NaN,NaN,Albania,2020-09-03 04:28:22,41.15330,20.168300,9728,296,5582,3850.0,Albania,338.036000,3.042763
2,NaN,NaN,NaN,Algeria,2020-09-03 04:28:22,28.03390,1.659600,45158,1523,31746,11889.0,Algeria,102.980447,3.372603
3,NaN,NaN,NaN,Andorra,2020-09-03 04:28:22,42.50630,1.521800,1199,53,909,237.0,Andorra,1551.802239,4.420350
4,NaN,NaN,NaN,Angola,2020-09-03 04:28:22,-11.20270,17.873900,2777,112,1115,1550.0,Angola,8.449393,4.033129


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,38054,38070,38113,38129,38140,38143,38162,38165,38196,38243
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,8605,8759,8927,9083,9195,9279,9380,9513,9606,9728
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,41858,42228,42619,43016,43403,43781,44146,44494,44833,45158


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1389,1397,1401,1401,1402,1402,1402,1402,1406,1409
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,254,259,263,266,271,275,280,284,290,296
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1446,1456,1465,1475,1483,1491,1501,1510,1518,1523


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,28360,28440,29042,29046,29059,29063,29089,29089,29231,29315
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,4413,4530,4633,4791,4923,5020,5139,5214,5441,5582
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,29369,29587,29886,30157,30436,30717,30978,31244,31493,31746


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1286,1286,1281,1284,1296,1309,1345,1348,1354,1345
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,4083,4100,4166,4199,4230,4307,4402,4422,4445,4458
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,633,628,616,614,620,624,628,628,629,617


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,22,22,22,22,22,22,22,22,23,23
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,32,32,33,34,35,36,36,38,38,38
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7,7,7,7,7,7,7,7,7,7


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
631,1001.0,Autauga,Alabama,US,2020-09-03 04:28:22,32.539527,-86.644082,1345,23,0,1322.0,"Autauga, Alabama, US",2407.417351,1.710037
668,1075.0,Lamar,Alabama,US,2020-09-03 04:28:22,33.779950,-88.096680,277,2,0,275.0,"Lamar, Alabama, US",2006.519377,0.722022
669,1077.0,Lauderdale,Alabama,US,2020-09-03 04:28:22,34.901719,-87.656247,1433,24,0,1409.0,"Lauderdale, Alabama, US",1545.363371,1.674808


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,6113510,185720,2231757
Brazil,3997865,123780,3387309
India,3853406,67376,2970492


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,6113510,185720,2231757,3696033,2020-09-03 04:28:22,37.923092,-91.345036
Brazil,3997865,123780,3387309,486776,2020-09-03 04:28:22,-12.669522,-48.480493
India,3853406,67376,2970492,815538,2020-09-03 04:28:22,23.088275,81.806127
Russia,1001965,17365,819043,165557,2020-09-03 04:28:22,54.546312,62.120860
Peru,657129,29068,480177,147884,2020-09-03 04:28:22,-10.637324,-75.277744


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,721281,13317,0
Texas,641855,13104,0
Florida,633442,11501,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,721281,13317,0,707964,2020-09-03 04:28:22,37.843962,-120.728594
Texas,641855,13104,0,628751,2020-09-03 04:28:22,31.660643,-98.653069
Florida,633442,11501,0,621941,2020-09-03 04:28:22,28.940755,-82.700744
New York,436218,32972,0,403246,2020-09-03 04:28:22,42.544151,-75.474183
Georgia,274613,5795,0,268818,2020-09-03 04:28:22,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,243935,5878,0
Florida,Miami-Dade,159400,2554,0
Arizona,Maricopa,134346,2995,0
Illinois,Cook,127616,5073,0
Texas,Harris,108085,2247,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,243935,5878,0,238057,2020-09-03 04:28:22,34.308284,-118.228241,6037.0
Florida,Miami-Dade,159400,2554,0,156846,2020-09-03 04:28:22,25.611236,-80.551706,12086.0
Arizona,Maricopa,134346,2995,0,131351,2020-09-03 04:28:22,33.348359,-112.491815,4013.0
Illinois,Cook,127616,5073,0,122543,2020-09-03 04:28:22,41.841448,-87.816588,17031.0
Texas,Harris,108085,2247,0,105838,2020-09-03 04:28:22,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-31,38165,9513,44494,1176,2654,94,417735,43781,25819,27438,...,338083,1595,41893,46728,1044,22729,10,1958,12097,6497
2020-09-01,38196,9606,44833,1184,2729,94,428239,43878,25923,27642,...,339415,1611,42127,47756,1044,23281,10,1962,12381,6559
2020-09-02,38243,9728,45158,1199,2777,94,439172,44075,26049,27969,...,340929,1626,42437,48883,1046,23875,10,1976,12415,6638


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-31,1402,284,1510,53,108,3,8660,879,657,733,...,41589,44,320,386,34,152,1,566,288,202
2020-09-01,1406,290,1518,53,109,3,8919,881,663,734,...,41592,44,323,391,34,159,1,570,290,203
2020-09-02,1409,296,1523,53,112,3,9118,884,678,734,...,41602,44,327,398,34,162,1,571,292,206


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-31,29089,5214,31244,908,1071,91,301195,37722,21348,23226,...,1682,1409,39275,38112,707,15056,8,1131,11469,5221
2020-09-01,29231,5441,31493,908,1084,91,308376,38356,21687,23565,...,1709,1419,39538,39027,735,15338,8,1160,11479,5241
2020-09-02,29315,5582,31746,909,1115,91,315530,38631,21690,23820,...,1742,1433,39664,39912,746,15483,8,1175,11494,5250


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1286,1286,1281,1284,1296,1309,1345,1348,1354,1345
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,261,264,266,266,266,268,271,273,277,277
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1339,1353,1364,1373,1384,1398,1405,1411,1424,1433
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,411,419,427,432,433,442,459,463,466,469
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,2940,2946,3006,3031,3091,3200,3247,3348,3395,3371


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-31,126058,5263,0,201835,61224,712475,57404,52879,17429,49,...,154933,632739,52107,1624,1134,120575,74635,10249,75603,3850
2020-09-01,127616,5297,0,202342,61497,715617,57755,53006,17535,49,...,156329,637721,52403,1635,1143,121593,74939,10513,76584,3874
2020-09-02,128239,5362,0,202861,62112,721281,57999,53108,17549,49,...,157831,641855,52822,1637,1144,122518,75377,10658,77129,3911


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-31        1348    4422     628  535   1034     537    837    2377   
2020-09-01        1354    4445     629  538   1045     541    840    2411   
2020-09-02        1345    4458     617  541   1036     540    839    2413   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-31          873      346  ...     164      7      149       48   
2020-09-01          874      353  ...     167      7      150       48   
2020-09-02          861      354  ...     170      7      154       48   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-31            302   430   298         73      108     11  
2020-09-01            302   430   301         73      108     11  
2020-09-02            304   433   303          0      109     19  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-31,2182,37,0,5029,797,13022,1945,4465,605,0,...,1754,12779,407,58,14,2580,1915,215,1122,37
2020-09-01,2200,39,0,5044,814,13150,1946,4466,605,0,...,1781,12919,409,58,15,2612,1931,224,1130,37
2020-09-02,2217,39,0,5065,841,13317,1952,4467,606,0,...,1797,13104,410,58,15,2641,1935,233,1142,41


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-31          22      38       7    7     11      13     36      30   
2020-09-01          23      38       7    7     11      13     36      32   
2020-09-02          23      38       7    7     11      13     36      32   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-31           39       11  ...       0      0        0        0   
2020-09-01           39       11  ...       0      0        0        0   
2020-09-02           39       11  ...       1      1        1        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-31              0     0     0         36        0      0  
2020-09-01              0     0     0         36        0      0  
2020-09-02              2     1     2          0        6      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-31,0.000079,0.014179,0.007883,0.046263,0.011433,0.0,0.022792,0.000709,0.002835,0.010013,...,0.004203,0.006309,0.011322,0.018749,0.003846,0.023644,0.0,0.002560,0.005988,0.013256
2020-09-01,0.000812,0.009776,0.007619,0.006803,0.028259,0.0,0.025145,0.002216,0.004028,0.007435,...,0.003940,0.010031,0.005586,0.022000,0.000000,0.024286,0.0,0.002043,0.023477,0.009543
2020-09-02,0.001230,0.012700,0.007249,0.012669,0.017589,0.0,0.025530,0.004490,0.004861,0.011830,...,0.004461,0.009311,0.007359,0.023599,0.001916,0.025514,0.0,0.007136,0.002746,0.012045


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-31,0.000000,0.014286,0.005996,0.0,0.009346,0.0,0.024004,0.002281,0.007669,0.000000,...,0.000072,0.0,0.022364,0.013123,0.0625,0.000000,0.0,0.003546,0.003484,0.030612
2020-09-01,0.002853,0.021127,0.005298,0.0,0.009259,0.0,0.029908,0.002275,0.009132,0.001364,...,0.000072,0.0,0.009375,0.012953,0.0000,0.046053,0.0,0.007067,0.006944,0.004950
2020-09-02,0.002134,0.020690,0.003294,0.0,0.027523,0.0,0.022312,0.003405,0.022624,0.000000,...,0.000240,0.0,0.012384,0.017903,0.0000,0.018868,0.0,0.001754,0.006897,0.014778


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-31,0.000000,0.014594,0.008587,0.006652,0.007526,0.0,0.024448,0.001912,0.011083,0.006762,...,0.035714,0.022496,0.016039,0.027527,0.017266,0.018192,0.0,0.007124,0.001310,0.031614
2020-09-01,0.004882,0.043537,0.007970,0.000000,0.012138,0.0,0.023842,0.016807,0.015880,0.014596,...,0.016052,0.007097,0.006696,0.024008,0.039604,0.018730,0.0,0.025641,0.000872,0.003831
2020-09-02,0.002874,0.025914,0.008034,0.001101,0.028598,0.0,0.023199,0.007170,0.000138,0.010821,...,0.019310,0.009866,0.003187,0.022677,0.014966,0.009454,0.0,0.012931,0.001307,0.001717


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-31,0.006572,0.007080,NaN,0.000863,0.006047,0.009241,0.003514,0.007315,0.004959,0.0,...,0.011873,0.005034,0.004879,0.004950,0.004429,0.007066,0.004238,0.013849,0.003531,0.005484
2020-09-01,0.012359,0.006460,NaN,0.002512,0.004459,0.004410,0.006115,0.002402,0.006082,0.0,...,0.009010,0.007874,0.005681,0.006773,0.007937,0.008443,0.004073,0.025759,0.012976,0.006234
2020-09-02,0.004882,0.012271,NaN,0.002565,0.010000,0.007915,0.004225,0.001924,0.000798,0.0,...,0.009608,0.006482,0.007996,0.001223,0.000875,0.007607,0.005845,0.013792,0.007116,0.009551


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-31      0.002230  0.004543  0.000000  0.003752  0.008780  0.001866   
2020-09-01      0.004451  0.005201  0.001592  0.005607  0.010638  0.007449   
2020-09-02     -0.006647  0.002925 -0.019078  0.005576 -0.008612 -0.001848   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-08-31      0.002395  0.013214  0.005760  0.002899  ...  0.025000    0.0   
2020-09-01      0.003584  0.014304  0.001145  0.020231  ...  0.018293    0.0   
2020-09-02     -0.001190  0.000830 -0.014874  0.002833  ...  0.017964    0.0   

Province_State                                                               \
Admin2          Sheridan Sublette Sweetwater     Teton     Uinta Unassigned   
2020-08-31      0.006757      0.0   0.003322  0.002331  0.010169        0.0   
2020-09-01      0.006711      0.0   0.000000  0.000000  0.010067        0.0   
2020-09-02      0.026667      0.0   0.006623  0.006977  0.006645       -1.0   

Province_State                      
Admin2          Washakie    Weston  
2020-08-31      0.000000  0.000000  
2020-09-01      0.000000  0.000000  
2020-09-02      0.009259  0.727273  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-31,0.009251,0.000000,NaN,-0.000199,0.016582,0.00657,0.001545,0.000000,0.001656,NaN,...,0.004007,0.007569,0.000000,0.0,0.000000,0.004282,0.005249,0.004673,0.000000,0.000000
2020-09-01,0.008249,0.054054,NaN,0.002983,0.021330,0.00983,0.000514,0.000224,0.000000,NaN,...,0.015393,0.010955,0.004914,0.0,0.071429,0.012403,0.008355,0.041860,0.007130,0.000000
2020-09-02,0.007727,0.000000,NaN,0.004163,0.033170,0.01270,0.003083,0.000224,0.001653,NaN,...,0.008984,0.014320,0.002445,0.0,0.000000,0.011103,0.002071,0.040179,0.010619,0.108108


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin Barbour      Bibb Blount Bullock Butler   
2020-08-31      0.000000  0.055556     0.0  0.166667    0.1     0.0    0.0   
2020-09-01      0.045455  0.000000     0.0  0.000000    0.0     0.0    0.0   
2020-09-02      0.000000  0.000000     0.0  0.000000    0.0     0.0    0.0   

Province_State                              ... Wyoming                  \
Admin2           Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2020-08-31      0.034483      0.0      0.0  ...     NaN    NaN      NaN   
2020-09-01      0.066667      0.0      0.0  ...     NaN    NaN      NaN   
2020-09-02      0.000000      0.0      0.0  ...     inf    inf      inf   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-31          NaN        NaN   NaN   NaN        0.0      NaN    NaN  
2020-09-01          NaN        NaN   NaN   NaN        0.0      NaN    NaN  
2020-09-02          inf        inf   inf   inf       -1.0      inf    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-31,0.000238,0.012865,0.008257,0.025241,0.020200,1.354073e-06,0.022427,0.002064,0.003325,0.009817,...,0.004322,0.007152,0.009972,0.020278,0.002513,0.023719,4.705739e-23,0.002844,0.008791,0.008810
2020-09-01,0.000525,0.011321,0.007938,0.016022,0.024230,6.770364e-07,0.023786,0.002140,0.003676,0.008626,...,0.004131,0.008592,0.007779,0.021139,0.001256,0.024003,2.352869e-23,0.002443,0.016134,0.009176
2020-09-02,0.000878,0.012010,0.007594,0.014345,0.020909,3.385182e-07,0.024658,0.003315,0.004268,0.010228,...,0.004296,0.008951,0.007569,0.022369,0.001586,0.024759,1.176435e-23,0.004789,0.009440,0.010610


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-31,0.000145,0.015593,0.006107,1.833989e-08,0.007581,1.446140e-42,0.019992,0.003596,0.026516,0.000009,...,0.000111,0.003113,0.020421,0.016330,0.041434,0.013624,0.0,0.002601,0.005129,0.021504
2020-09-01,0.001499,0.018360,0.005703,9.169945e-09,0.008420,7.230700e-43,0.024950,0.002935,0.017824,0.000687,...,0.000092,0.001556,0.014898,0.014642,0.020717,0.029839,0.0,0.004834,0.006037,0.013227
2020-09-02,0.001816,0.019525,0.004498,4.584973e-09,0.017971,3.615350e-43,0.023631,0.003170,0.020224,0.000343,...,0.000166,0.000778,0.013641,0.016272,0.010359,0.024353,0.0,0.003294,0.006467,0.014003


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-31,0.000673,0.019220,0.008718,0.004251,0.008966,0.005622,0.023942,0.003450,0.011618,0.008447,...,0.021030,0.016410,0.014152,0.028066,0.022038,0.017691,2.757269e-25,0.007181,0.012315,0.017214
2020-09-01,0.002777,0.031378,0.008344,0.002125,0.010552,0.002811,0.023892,0.010129,0.013749,0.011521,...,0.018541,0.011754,0.010424,0.026037,0.030821,0.018211,1.378634e-25,0.016411,0.006593,0.010522
2020-09-02,0.002825,0.028646,0.008189,0.001613,0.019575,0.001406,0.023545,0.008649,0.006944,0.011171,...,0.018925,0.010810,0.006806,0.024357,0.022893,0.013832,6.893172e-26,0.014671,0.003950,0.006120


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-31,0.009195,0.010199,NaN,0.001621,0.008251,0.007839,0.003976,0.004006,0.004772,9.098773e-52,...,0.009910,0.006125,0.006703,0.005861,0.012463,0.007876,0.005532,0.014157,0.006342,0.006678
2020-09-01,0.010777,0.008329,NaN,0.002067,0.006355,0.006125,0.005046,0.003204,0.005427,4.549387e-52,...,0.009460,0.006999,0.006192,0.006317,0.010200,0.008159,0.004802,0.019958,0.009659,0.006456
2020-09-02,0.007830,0.010300,NaN,0.002316,0.008178,0.007020,0.004635,0.002564,0.003113,2.274693e-52,...,0.009534,0.006741,0.007094,0.003770,0.005537,0.007883,0.005324,0.016875,0.008388,0.008003


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-31      0.009920  0.011101  0.002605  0.008592  0.013164  0.002972   
2020-09-01      0.007186  0.008151  0.002099  0.007100  0.011901  0.005210   
2020-09-02      0.000269  0.005538 -0.008490  0.006338  0.001644  0.001681   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-08-31      0.005945  0.014593  0.003822  0.006625  ...  0.013530   
2020-09-01      0.004765  0.014448  0.002484  0.013428  ...  0.015911   
2020-09-02      0.001787  0.007639 -0.006195  0.008130  ...  0.016938   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-08-31      0.000011  0.018125  0.000168   0.006246  0.005158  0.008636   
2020-09-01      0.000005  0.012418  0.000084   0.003123  0.002579  0.009351   
2020-09-02      0.000003  0.019542  0.000042   0.004873  0.004778  0.007998   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-08-31       0.050314  0.000037 -0.000176  
2020-09-01       0.024346  0.000019 -0.000088  
2020-09-02      -0.495957  0.004639  0.363637  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-31,0.010016,0.001525,NaN,0.003009,0.017319,0.006320,0.001399,0.000020,0.000871,NaN,...,0.008962,0.007905,0.001139,2.635027e-10,0.003537,0.004076,0.003384,0.012816,0.001965,0.000059
2020-09-01,0.009133,0.027790,NaN,0.002996,0.019324,0.008075,0.000957,0.000122,0.000436,NaN,...,0.012177,0.009430,0.003026,1.317514e-10,0.037483,0.008239,0.005869,0.027338,0.004548,0.000030
2020-09-02,0.008430,0.013895,NaN,0.003580,0.026247,0.010387,0.002020,0.000173,0.001044,NaN,...,0.010581,0.011875,0.002736,6.587568e-11,0.018741,0.009671,0.003970,0.033758,0.007584,0.054069


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                          \
Admin2               Autauga   Baldwin   Barbour      Bibb    Blount   
2020-08-31      2.307906e-08  0.034649  0.000020  0.105778  0.098337   
2020-09-01      2.272728e-02  0.017324  0.000010  0.052889  0.049168   
2020-09-02      1.136364e-02  0.008662  0.000005  0.026445  0.024584   

Province_State                                                        ...  \
Admin2           Bullock        Butler   Calhoun  Chambers  Cherokee  ...   
2020-08-31      0.000006  1.703453e-09  0.031154  0.001645  0.006975  ...   
2020-09-01      0.000003  8.517265e-10  0.048911  0.000822  0.003487  ...   
2020-09-02      0.000001  4.258633e-10  0.024455  0.000411  0.001744  ...   

Province_State Wyoming                                                  \
Admin2            Park Platte Sheridan Sublette Sweetwater Teton Uinta   
2020-08-31         NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-09-01         NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-09-02         NaN    NaN      NaN      NaN        NaN   NaN   NaN   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-08-31       0.000061      NaN    NaN  
2020-09-01       0.000031      NaN    NaN  
2020-09-02      -0.499985      NaN    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200902,AK,6233,370651,NaN,46.0,NaN,NaN,NaN,10.0,...,376884,0,0,0eaef7dbd992985ab0f8d68e7290a358f8b6e44f,0,0,0,0,0,NaN
1,20200902,AL,128239,835994,NaN,959.0,14753.0,NaN,1501.0,NaN,...,964233,17,215,fca39761bbc6be0099b27c6b880230afcfeabc1b,0,0,0,0,0,NaN
2,20200902,AR,62112,675337,NaN,435.0,4341.0,NaN,NaN,90.0,...,737449,27,35,3c318be701fbb9138ced6d0286e3874d5edf35fe,0,0,0,0,0,NaN
3,20200902,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,48ebb399ca7a84fe4e65df7be64d01c2adde9b27,0,0,0,0,0,NaN
4,20200902,AZ,202861,1011066,NaN,752.0,21449.0,248.0,NaN,146.0,...,1213927,21,44,36685dfe649f351d4b5be90630ede6819fded584,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200902,AK,6233.0,370651.0,NaN,46.0,NaN,NaN,NaN,10.0,...,376884,0,0,0eaef7dbd992985ab0f8d68e7290a358f8b6e44f,0,0,0,0,0,NaN
1,20200902,AL,128239.0,835994.0,NaN,959.0,14753.0,NaN,1501.0,NaN,...,964233,17,215,fca39761bbc6be0099b27c6b880230afcfeabc1b,0,0,0,0,0,NaN
2,20200902,AR,62112.0,675337.0,NaN,435.0,4341.0,NaN,NaN,90.0,...,737449,27,35,3c318be701fbb9138ced6d0286e3874d5edf35fe,0,0,0,0,0,NaN
3,20200902,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,48ebb399ca7a84fe4e65df7be64d01c2adde9b27,0,0,0,0,0,NaN
4,20200902,AZ,202861.0,1011066.0,NaN,752.0,21449.0,248.0,NaN,146.0,...,1213927,21,44,36685dfe649f351d4b5be90630ede6819fded584,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-09-02,AK,6233,370651,NaN,46.0,NaN,NaN,NaN,10.0,NaN,...,376884,0,0,0eaef7dbd992985ab0f8d68e7290a358f8b6e44f,0,0,0,0,0,NaN
2020-09-02,AL,128239,835994,NaN,959.0,14753.0,NaN,1501.0,NaN,827.0,...,964233,17,215,fca39761bbc6be0099b27c6b880230afcfeabc1b,0,0,0,0,0,NaN
2020-09-02,AR,62112,675337,NaN,435.0,4341.0,NaN,NaN,90.0,556.0,...,737449,27,35,3c318be701fbb9138ced6d0286e3874d5edf35fe,0,0,0,0,0,NaN
2020-09-02,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,48ebb399ca7a84fe4e65df7be64d01c2adde9b27,0,0,0,0,0,NaN
2020-09-02,AZ,202861,1011066,NaN,752.0,21449.0,248.0,NaN,146.0,NaN,...,1213927,21,44,36685dfe649f351d4b5be90630ede6819fded584,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-09-02,AK,6233.0,370651.0,NaN,46.0,NaN,NaN,NaN,10.0,NaN,...,376884,0,0,0eaef7dbd992985ab0f8d68e7290a358f8b6e44f,0,0,0,0,0,NaN
2020-09-02,AL,128239.0,835994.0,NaN,959.0,14753.0,NaN,1501.0,NaN,827.0,...,964233,17,215,fca39761bbc6be0099b27c6b880230afcfeabc1b,0,0,0,0,0,NaN
2020-09-02,AR,62112.0,675337.0,NaN,435.0,4341.0,NaN,NaN,90.0,556.0,...,737449,27,35,3c318be701fbb9138ced6d0286e3874d5edf35fe,0,0,0,0,0,NaN
2020-09-02,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,48ebb399ca7a84fe4e65df7be64d01c2adde9b27,0,0,0,0,0,NaN
2020-09-02,AZ,202861.0,1011066.0,NaN,752.0,21449.0,248.0,NaN,146.0,NaN,...,1213927,21,44,36685dfe649f351d4b5be90630ede6819fded584,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE